In [10]:
#Use the same imports 
import tensorflow as tf

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc 
rc('text', usetex=False) # switch tex off for using it in the ipython notebook 
import scipy.io
from scipy.interpolate import griddata
from pyDOE import lhs
#from plotting import newfig, savefig
from mpl_toolkits.mplot3d import Axes3D
import time
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [12]:
from Schrodinger import DeepHPM

### Define Domain Bounds

In [13]:
lb_idn = np.array([0.0, -5.0])
ub_idn = np.array([np.pi/2, 5.0])

lb_sol = np.array([0.0, -5.0])
ub_sol = np.array([np.pi/2, 5.0])

### Load Data

In [14]:
data_idn = scipy.io.loadmat('../Data/NLS.mat')

t_idn = data_idn['t'].flatten()[:,None]
x_idn = data_idn['x'].flatten()[:,None]
Exact_u_idn = np.real(data_idn['usol'])
Exact_v_idn = np.imag(data_idn['usol'])

T_idn, X_idn = np.meshgrid(t_idn,x_idn)

keep = 1
index = int(keep*t_idn.shape[0])
T_idn = T_idn[:,0:index]
X_idn = X_idn[:,0:index]
Exact_u_idn = Exact_u_idn[:,0:index]
Exact_v_idn = Exact_v_idn[:,0:index]
Exact_uv_idn = np.sqrt(Exact_u_idn**2 + Exact_v_idn**2)    

t_idn_star = T_idn.flatten()[:,None]
x_idn_star = X_idn.flatten()[:,None]
X_idn_star = np.hstack((t_idn_star, x_idn_star))
u_idn_star = Exact_u_idn.flatten()[:,None]
v_idn_star = Exact_v_idn.flatten()[:,None]

#

data_sol = scipy.io.loadmat('../Data/NLS.mat')

t_sol = data_sol['t'].flatten()[:,None]
x_sol = data_sol['x'].flatten()[:,None]
Exact_u_sol = np.real(data_sol['usol'])
Exact_v_sol = np.imag(data_sol['usol'])
Exact_uv_sol = np.sqrt(Exact_u_sol**2 + Exact_v_sol**2)

T_sol, X_sol = np.meshgrid(t_sol,x_sol)

t_sol_star = T_sol.flatten()[:,None]
x_sol_star = X_sol.flatten()[:,None]
X_sol_star = np.hstack((t_sol_star, x_sol_star))
u_sol_star = Exact_u_sol.flatten()[:,None]
v_sol_star = Exact_v_sol.flatten()[:,None]

###  Prepare Training Data

#### For identification

In [15]:
N_train = 10000

idx = np.random.choice(t_idn_star.shape[0], N_train, replace=False)    
t_train = t_idn_star[idx,:]
x_train = x_idn_star[idx,:]
u_train = u_idn_star[idx,:]
v_train = v_idn_star[idx,:]

noise = 0.00
u_train = u_train + noise*np.std(u_train)*np.random.randn(u_train.shape[0], u_train.shape[1])
v_train = v_train + noise*np.std(v_train)*np.random.randn(v_train.shape[0], v_train.shape[1])

#### For solution 

In [16]:
N0 = Exact_u_sol.shape[0]
N_b = Exact_u_sol.shape[1]
N_f = 20000

idx_x = np.random.choice(x_sol.shape[0], N0, replace=False)
x0_train = x_sol[idx_x,:]
u0_train = Exact_u_sol[idx_x,0:1]
v0_train = Exact_v_sol[idx_x,0:1]

idx_t = np.random.choice(t_sol.shape[0], N_b, replace=False)
tb_train = t_sol[idx_t,:]

X_f_train = lb_sol + (ub_sol-lb_sol)*lhs(2, N_f)

### Define the Model 

#### Layers

In [17]:
uv_layers = [2, 50, 50, 50, 50, 1]
pde_layers = [4, 100, 100, 1]
layers = [2, 50, 50, 50, 50, 1]
print(pde_layers[0])

4


#### Create Model

In [18]:
model = DeepHPM(t_train, x_train, u_train, v_train,
                x0_train, u0_train, v0_train, tb_train, X_f_train,
                uv_layers, pde_layers,
                layers,
                lb_idn, ub_idn,
                lb_sol, ub_sol)
    

(?, 4)
(?, 4)
(?, 4)


### Training

#### Training the identifiers

In [ ]:
model.idn_uv_train(N_iter=0)
model.idn_fg_train(N_iter=0)



Loss: 7.853013e+03
Loss: 3.813241e+04
Loss: 6.225541e+03
Loss: 6.200783e+03
Loss: 6.176642e+03
Loss: 6.175746e+03
Loss: 6.175332e+03
Loss: 6.175206e+03
Loss: 6.174310e+03
Loss: 6.172842e+03
Loss: 6.167245e+03
Loss: 6.043319e+03
Loss: 2.798502e+04
Loss: 6.041661e+03
Loss: 6.017544e+03
Loss: 5.946670e+03
Loss: 5.945031e+03
Loss: 5.822103e+03
Loss: 5.779811e+03
Loss: 5.679379e+03
Loss: 5.539029e+03
Loss: 5.482560e+03
Loss: 5.460147e+03
Loss: 5.438690e+03
Loss: 5.404603e+03
Loss: 5.406287e+03
Loss: 5.401524e+03
Loss: 5.400406e+03
Loss: 5.398998e+03
Loss: 5.395457e+03
Loss: 5.384441e+03
Loss: 5.370136e+03
Loss: 5.352408e+03
Loss: 5.323854e+03
Loss: 5.296637e+03
Loss: 5.256372e+03
Loss: 5.229310e+03
Loss: 5.187571e+03
Loss: 5.157149e+03
Loss: 5.133878e+03
Loss: 5.113650e+03
Loss: 5.094680e+03
Loss: 5.068039e+03
Loss: 5.045444e+03
Loss: 5.019866e+03
Loss: 4.998220e+03
Loss: 4.983969e+03
Loss: 4.978174e+03
Loss: 4.972496e+03
Loss: 4.966348e+03
Loss: 4.948780e+03
Loss: 4.918496e+03
Loss: 4.8677

Loss: 9.370768e-01
Loss: 9.304638e-01
Loss: 9.234851e-01
Loss: 9.174365e-01
Loss: 9.101307e-01
Loss: 8.980125e-01
Loss: 8.915927e-01
Loss: 8.884385e-01
Loss: 8.857276e-01
Loss: 8.798382e-01
Loss: 8.700271e-01
Loss: 8.631901e-01
Loss: 8.567495e-01
Loss: 8.493754e-01
Loss: 8.393305e-01
Loss: 8.341824e-01
Loss: 8.232119e-01
Loss: 8.184741e-01
Loss: 8.149699e-01
Loss: 8.108527e-01
Loss: 8.121825e-01
Loss: 8.066679e-01
Loss: 8.006257e-01
Loss: 7.961983e-01
Loss: 7.931837e-01
Loss: 7.878425e-01
Loss: 7.956822e-01
Loss: 7.853265e-01
Loss: 7.802435e-01
Loss: 7.756301e-01
Loss: 7.709951e-01
Loss: 7.638296e-01
Loss: 7.545115e-01
Loss: 7.538996e-01
Loss: 7.499112e-01
Loss: 7.432745e-01
Loss: 7.372476e-01
Loss: 7.291046e-01
Loss: 7.210181e-01
Loss: 7.158288e-01
Loss: 7.112797e-01
Loss: 7.077333e-01
Loss: 7.045298e-01
Loss: 7.010764e-01
Loss: 6.973284e-01
Loss: 6.952058e-01
Loss: 6.919533e-01
Loss: 6.891284e-01
Loss: 6.863010e-01
Loss: 6.824980e-01
Loss: 6.750151e-01
Loss: 8.481252e-01
Loss: 6.7412

Loss: 1.792844e-01
Loss: 1.787225e-01
Loss: 1.783752e-01
Loss: 1.780943e-01
Loss: 1.778424e-01
Loss: 1.775725e-01
Loss: 1.772375e-01
Loss: 1.769247e-01
Loss: 1.764140e-01
Loss: 1.761536e-01
Loss: 1.752309e-01
Loss: 1.786623e-01
Loss: 1.750326e-01
Loss: 1.744997e-01
Loss: 1.739171e-01
Loss: 1.730385e-01
Loss: 1.725465e-01
Loss: 1.721934e-01
Loss: 1.719193e-01
Loss: 1.715702e-01
Loss: 1.710316e-01
Loss: 1.707122e-01
Loss: 1.704358e-01
Loss: 1.698517e-01
Loss: 1.692539e-01
Loss: 1.685521e-01
Loss: 1.683214e-01
Loss: 1.676204e-01
Loss: 1.673753e-01
Loss: 1.670807e-01
Loss: 1.666127e-01
Loss: 1.670285e-01
Loss: 1.662786e-01
Loss: 1.657040e-01
Loss: 1.653576e-01
Loss: 1.648819e-01
Loss: 1.642667e-01
Loss: 1.644380e-01
Loss: 1.639811e-01
Loss: 1.636886e-01
Loss: 1.634774e-01
Loss: 1.630883e-01
Loss: 1.624881e-01
Loss: 1.624866e-01
Loss: 1.621682e-01
Loss: 1.617863e-01
Loss: 1.615572e-01
Loss: 1.613671e-01
Loss: 1.610255e-01
Loss: 1.606019e-01
Loss: 1.602454e-01
Loss: 1.596355e-01
Loss: 1.5921

Loss: 8.761171e-02
Loss: 8.749767e-02
Loss: 8.735111e-02
Loss: 8.715201e-02
Loss: 8.698972e-02
Loss: 8.687964e-02
Loss: 8.675344e-02
Loss: 8.662053e-02
Loss: 8.653610e-02
Loss: 8.633802e-02
Loss: 8.625342e-02
Loss: 8.618528e-02
Loss: 8.604876e-02
Loss: 8.628421e-02
Loss: 8.596828e-02
Loss: 8.579400e-02
Loss: 8.570766e-02
Loss: 8.560164e-02
Loss: 8.602068e-02
Loss: 8.556587e-02
Loss: 8.547267e-02
Loss: 8.537014e-02
Loss: 8.519848e-02
Loss: 8.506174e-02
Loss: 8.501838e-02
Loss: 8.482177e-02
Loss: 8.474979e-02
Loss: 8.464593e-02
Loss: 8.456524e-02
Loss: 8.443647e-02
Loss: 8.436017e-02
Loss: 8.427411e-02
Loss: 8.415829e-02
Loss: 8.415861e-02
Loss: 8.407196e-02
Loss: 8.395001e-02
Loss: 8.385488e-02
Loss: 8.375951e-02
Loss: 8.359189e-02
Loss: 8.362272e-02
Loss: 8.350304e-02
Loss: 8.334163e-02
Loss: 8.327332e-02
Loss: 8.312270e-02
Loss: 8.293501e-02
Loss: 8.366121e-02
Loss: 8.287209e-02
Loss: 8.268902e-02
Loss: 8.261991e-02
Loss: 8.255897e-02
Loss: 8.248274e-02
Loss: 8.234335e-02
Loss: 8.2224

Loss: 5.537437e-02
Loss: 5.580376e-02
Loss: 5.534811e-02
Loss: 5.528629e-02
Loss: 5.522930e-02
Loss: 5.517769e-02
Loss: 5.510011e-02
Loss: 5.497462e-02
Loss: 5.496887e-02
Loss: 5.490115e-02
Loss: 5.481622e-02
Loss: 5.474160e-02
Loss: 5.464032e-02
Loss: 5.454204e-02
Loss: 5.447172e-02
Loss: 5.442912e-02
Loss: 5.437321e-02
Loss: 5.430609e-02
Loss: 5.419905e-02
Loss: 5.412098e-02
Loss: 5.398214e-02
Loss: 5.394540e-02
Loss: 5.387987e-02
Loss: 5.381573e-02
Loss: 5.376028e-02
Loss: 5.374766e-02
Loss: 5.370017e-02
Loss: 5.367793e-02
Loss: 5.364625e-02
Loss: 5.359080e-02
Loss: 5.353632e-02
Loss: 5.342804e-02
Loss: 5.337735e-02
Loss: 5.329977e-02
Loss: 5.328860e-02
Loss: 5.322814e-02
Loss: 5.318581e-02
Loss: 5.314498e-02
Loss: 5.308691e-02
Loss: 5.307811e-02
Loss: 5.303228e-02
Loss: 5.297537e-02
Loss: 5.293127e-02
Loss: 5.289701e-02
Loss: 5.285656e-02
Loss: 5.280306e-02
Loss: 5.276700e-02
Loss: 5.268621e-02
Loss: 5.259968e-02
Loss: 5.256862e-02
Loss: 5.247586e-02
Loss: 5.250080e-02
Loss: 5.2449

...

Loss: 1.275046e+01
Loss: 1.274909e+01
Loss: 1.274860e+01
Loss: 1.274744e+01
Loss: 1.274685e+01
Loss: 1.274617e+01
Loss: 1.274509e+01
Loss: 1.274542e+01
Loss: 1.274408e+01
Loss: 1.274260e+01
Loss: 1.274185e+01
Loss: 1.274092e+01
Loss: 1.273939e+01
Loss: 1.273803e+01
Loss: 1.273700e+01
Loss: 1.273609e+01
Loss: 1.273538e+01
Loss: 1.273458e+01
Loss: 1.273270e+01
Loss: 1.273215e+01
Loss: 1.273145e+01
Loss: 1.273090e+01
Loss: 1.273021e+01
Loss: 1.272987e+01
Loss: 1.272875e+01
Loss: 1.272798e+01
Loss: 1.272724e+01
Loss: 1.272664e+01
Loss: 1.272588e+01
Loss: 1.272537e+01
Loss: 1.272481e+01
Loss: 1.272429e+01
Loss: 1.272359e+01
Loss: 1.272287e+01
Loss: 1.272204e+01
Loss: 1.272180e+01
Loss: 1.272120e+01
Loss: 1.272048e+01
Loss: 1.271995e+01
Loss: 1.271927e+01
Loss: 1.271803e+01
Loss: 1.271745e+01
Loss: 1.271625e+01
Loss: 1.271571e+01
Loss: 1.271496e+01
Loss: 1.271355e+01
Loss: 1.271648e+01
Loss: 1.271302e+01
Loss: 1.271175e+01
Loss: 1.271118e+01
Loss: 1.271051e+01
Loss: 1.270976e+01
Loss: 1.2708

Loss: 1.240805e+01
Loss: 1.240730e+01
Loss: 1.240661e+01
Loss: 1.240583e+01
Loss: 1.240526e+01
Loss: 1.240447e+01
Loss: 1.240370e+01
Loss: 1.240227e+01
Loss: 1.240161e+01
Loss: 1.240103e+01
Loss: 1.240028e+01
Loss: 1.239977e+01
Loss: 1.239843e+01
Loss: 1.239995e+01
Loss: 1.239801e+01
Loss: 1.239719e+01
Loss: 1.239665e+01
Loss: 1.239577e+01
Loss: 1.239484e+01
Loss: 1.239429e+01
Loss: 1.239360e+01
Loss: 1.239271e+01
Loss: 1.239179e+01
Loss: 1.239037e+01
Loss: 1.238877e+01
Loss: 1.238858e+01
Loss: 1.238692e+01
Loss: 1.238650e+01
Loss: 1.238562e+01
Loss: 1.238406e+01
Loss: 1.238296e+01
Loss: 1.238165e+01
Loss: 1.238085e+01
Loss: 1.237989e+01
Loss: 1.237835e+01
Loss: 1.237753e+01
Loss: 1.237643e+01
Loss: 1.237586e+01
Loss: 1.237524e+01
Loss: 1.237357e+01
Loss: 1.237283e+01
Loss: 1.237073e+01
Loss: 1.237035e+01
Loss: 1.236946e+01
Loss: 1.236898e+01
Loss: 1.236837e+01
Loss: 1.236776e+01
Loss: 1.236699e+01
Loss: 1.236651e+01
Loss: 1.236561e+01
Loss: 1.236476e+01
Loss: 1.236307e+01
Loss: 1.2362

Loss: 1.202974e+01
Loss: 1.202895e+01
Loss: 1.202832e+01
Loss: 1.202771e+01
Loss: 1.202705e+01
Loss: 1.202623e+01
Loss: 1.202661e+01
Loss: 1.202568e+01
Loss: 1.202507e+01
Loss: 1.202427e+01
Loss: 1.202389e+01
Loss: 1.202308e+01
Loss: 1.202245e+01
Loss: 1.202205e+01
Loss: 1.202162e+01
Loss: 1.202105e+01
Loss: 1.202005e+01
Loss: 1.201958e+01
Loss: 1.201869e+01
Loss: 1.201815e+01
Loss: 1.201772e+01
Loss: 1.201718e+01
Loss: 1.201660e+01
Loss: 1.201645e+01
Loss: 1.201554e+01
Loss: 1.201514e+01
Loss: 1.201443e+01
Loss: 1.201367e+01
Loss: 1.201293e+01
Loss: 1.201214e+01
Loss: 1.201153e+01
Loss: 1.201100e+01
Loss: 1.200977e+01
Loss: 1.201097e+01
Loss: 1.200924e+01
Loss: 1.200824e+01
Loss: 1.200728e+01
Loss: 1.200657e+01
Loss: 1.200580e+01
Loss: 1.200522e+01
Loss: 1.200397e+01
Loss: 1.200329e+01
Loss: 1.200140e+01
Loss: 1.200350e+01
Loss: 1.200074e+01
Loss: 1.199954e+01
Loss: 1.199891e+01
Loss: 1.199816e+01
Loss: 1.199766e+01
Loss: 1.199695e+01
Loss: 1.199665e+01
Loss: 1.199607e+01
Loss: 1.1995

Loss: 1.167463e+01
Loss: 1.167314e+01
Loss: 1.167228e+01
Loss: 1.167088e+01
Loss: 1.167019e+01
Loss: 1.166988e+01
Loss: 1.166926e+01
Loss: 1.166862e+01
Loss: 1.166804e+01
Loss: 1.166686e+01
Loss: 1.166916e+01
Loss: 1.166638e+01
Loss: 1.166509e+01
Loss: 1.166431e+01
Loss: 1.166285e+01
Loss: 1.166397e+01
Loss: 1.166245e+01
Loss: 1.166158e+01
Loss: 1.166018e+01
Loss: 1.165928e+01
Loss: 1.165856e+01
Loss: 1.165806e+01
Loss: 1.165728e+01
Loss: 1.165639e+01
Loss: 1.165645e+01
Loss: 1.165569e+01
Loss: 1.165490e+01
Loss: 1.165447e+01
Loss: 1.165401e+01
Loss: 1.165265e+01
Loss: 1.165222e+01
Loss: 1.165102e+01
Loss: 1.165062e+01
Loss: 1.164975e+01
Loss: 1.164812e+01
Loss: 1.165023e+01
Loss: 1.164754e+01
Loss: 1.164650e+01
Loss: 1.164573e+01
Loss: 1.164461e+01
Loss: 1.164366e+01
Loss: 1.164253e+01
Loss: 1.164166e+01
Loss: 1.164089e+01
Loss: 1.164002e+01
Loss: 1.163816e+01
Loss: 1.163924e+01
Loss: 1.163747e+01
Loss: 1.163684e+01
Loss: 1.163607e+01
Loss: 1.163527e+01
Loss: 1.163435e+01
Loss: 1.1633

Loss: 1.133081e+01
Loss: 1.133027e+01
Loss: 1.132976e+01
Loss: 1.132903e+01
Loss: 1.132830e+01
Loss: 1.132784e+01
Loss: 1.132688e+01
Loss: 1.133180e+01
Loss: 1.132648e+01
Loss: 1.132535e+01
Loss: 1.132434e+01
Loss: 1.132319e+01
Loss: 1.132275e+01
Loss: 1.132180e+01
Loss: 1.132121e+01
Loss: 1.132050e+01
Loss: 1.131953e+01
Loss: 1.131938e+01
Loss: 1.131767e+01
Loss: 1.131726e+01
Loss: 1.131647e+01
Loss: 1.131586e+01
Loss: 1.131513e+01
Loss: 1.131422e+01
Loss: 1.131345e+01
Loss: 1.131273e+01
Loss: 1.131189e+01
Loss: 1.131135e+01
Loss: 1.131061e+01
Loss: 1.130906e+01
Loss: 1.130865e+01
Loss: 1.130663e+01
Loss: 1.130601e+01
Loss: 1.130531e+01
Loss: 1.130462e+01
Loss: 1.130360e+01
Loss: 1.130260e+01
Loss: 1.130154e+01
Loss: 1.130106e+01
Loss: 1.130000e+01
Loss: 1.129871e+01
Loss: 1.129692e+01
Loss: 1.129601e+01
Loss: 1.129499e+01
Loss: 1.129462e+01
Loss: 1.129385e+01
Loss: 1.129389e+01
Loss: 1.129315e+01
Loss: 1.129200e+01
Loss: 1.129137e+01
Loss: 1.129082e+01
Loss: 1.128992e+01
Loss: 1.1288

This two lines trains the two identifier networks. The number of iterations is set to zero means there is no adam iteration

#### Training the solver

In [ ]:
model.sol_train(N_iter=0)

In [ ]:
u_pred, v_pred, f_pred, g_pred = model.sol_predict(t_sol_star, x_sol_star)

u_pred_idn, v_pred_idn, f_pred_idn, g_pred_idn = model.sol_predict(t_idn_star, x_idn_star)

uv_pred = np.sqrt(u_pred**2 + v_pred**2)
uv_sol_star = np.sqrt(u_sol_star**2 + v_sol_star**2)
error_u = np.linalg.norm(u_sol_star-u_pred,2)/np.linalg.norm(u_sol_star,2)
error_v = np.linalg.norm(v_sol_star-v_pred,2)/np.linalg.norm(v_sol_star,2)
error_uv = np.linalg.norm(uv_sol_star-uv_pred,2)/np.linalg.norm(uv_sol_star,2)
print('Error uv: %e' % (error_uv))

error_u_idn = np.linalg.norm(u_idn_star-u_pred_idn,2)/np.linalg.norm(u_idn_star,2)
error_v_idn = np.linalg.norm(v_idn_star-v_pred_idn,2)/np.linalg.norm(v_idn_star,2)

print('Error u: %e' % (error_u))
print('Error v: %e' % (error_v))
print('Error u (idn): %e' % (error_u_idn))
print('Error v (idn): %e' % (error_v_idn))

U_pred = griddata(X_sol_star, u_pred.flatten(), (T_sol, X_sol), method='cubic')
V_pred = griddata(X_sol_star, v_pred.flatten(), (T_sol, X_sol), method='cubic')
UV_pred = np.sqrt(U_pred**2 + V_pred**2)

In [ ]:
from matplotlib import rc 
rc('text', usetex=False) # switch tex off for using it in the ipython notebook 
plt.figure(figsize=(10,10))
fig = plt.gcf()
ax = plt.gca()
gs = gridspec.GridSpec(1, 2)
gs.update(top=0.8, bottom=0.2, left=0.1, right=0.9, wspace=0.5)
ax = plt.subplot(gs[:, 0])
h = ax.imshow(Exact_u_idn, interpolation='nearest', cmap='jet', 
              extent=[lb_sol[0], ub_sol[0], lb_sol[1], ub_sol[1]],
              origin='lower', aspect='auto')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)

fig.colorbar(h, cax=cax)
ax.set_xlabel('t')
ax.set_ylabel('x')
ax.set_title('Exact Dynamics', fontsize = 10)

########     Exact p(t,x,y)     ########### 
ax = plt.subplot(gs[:, 1])
h = ax.imshow(u_pred, interpolation='nearest', cmap='jet', 
              extent=[lb_sol[0], ub_sol[0], lb_sol[1], ub_sol[1]], 
              origin='lower', aspect='auto')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)

fig.colorbar(h, cax=cax)
ax.set_xlabel('t')
ax.set_ylabel('x')
ax.set_title('Learned Dynamics', fontsize = 10)

In [ ]:
from matplotlib import colors
plt.figure(figsize=(10,10))
difference = np.abs(Exact_uv_sol - UV_pred)
plt.imshow(np.abs(Exact_uv_sol - UV_pred),interpolation='nearest')
plt.colorbar()

#### Saving process

In [ ]:
import h5py
#creating the file 
with h5py.File("fourth_degree_deep_hpm.hdf5", "w") as f:
    group = 'first_degree_gradients'
    ds_pred = f.create_dataset(group + '/prediction',data = UV_pred)
    ds_exact = f.create_dataset(group + '/exact',data = Exact_uv_sol)
    ds_difference = f.create_dataset(group + '/diff',data = difference)
    ds_pred.attrs['error_u'] = error_u
    ds_pred.attrs['error_v'] = error_v




In [ ]:
print("Hallo)")